In [ ]:
import pandas as pd
from pyhive import hive

def execute_query(cursor, query, fetch=False):
    try:
        cursor.execute(query)
        if fetch:
            return cursor.fetchall()
    except Exception as e:
        print(e)
        return e

conn = hive.Connection(host="localhost", port=10000, username="hive")
cursor = conn.cursor()

execute_query(cursor, '''
SHOW DATABASES
''', True)

In [12]:
execute_query(cursor, '''
CREATE DATABASE IF NOT EXISTS course
''')

execute_query(cursor, '''
USE course
''')

execute_query(cursor, '''
CREATE TABLE IF NOT EXISTS my_table (
    col1 STRING,
    col2 ARRAY<STRING>,
    col3 STRING,
    col4 INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '.'
COLLECTION ITEMS TERMINATED BY ':'
LINES TERMINATED BY '\\n'
STORED AS TEXTFILE
''')

### Activity 1
1. Study the structure of data then use create statement to create a table with necessary number of columns.
2. Load the data into table.
3. View data in tables after loading.

In [48]:
execute_query(cursor, '''
SET hive.exec.dynamic.partition=true
''',False)

execute_query(cursor, '''
SET hive.exec.dynamic.partition.mode=nonstrict
''',False)

In [ ]:
execute_query(cursor, '''
CREATE TABLE IF NOT EXISTS orders (
    cust_id VARCHAR(20),
    cust_name VARCHAR(50),
    odr_date STRING,
    ship_date STRING,
    courier VARCHAR(20),
    recvd_date STRING,
    returned VARCHAR(5),
    returned_date STRING,
    return_reason VARCHAR(50)
) 
PARTITIONED BY (cust_name STRING, returned STRING)  
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY "," 
LINES TERMINATED BY "\n" 
STORED AS TEXTFILE
TBLPROPERTIES("skip.header.line.count"="1");
''')

In [14]:
pd.DataFrame(execute_query(cursor, '''
SELECT * FROM orders
''', True))

""


In [ ]:
pd.DataFrame(execute_query(cursor, '''
SET -v
''', True))

In [56]:
execute_query(cursor, '''
select * from orders
''',True)

In [58]:
execute_query(cursor, '''
LOAD DATA LOCAL INPATH '/opt/hive/assignment_create_table_2018.txt' 
INTO TABLE orders;
''')

TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['Server-side error; please check HS2 logs.'], sqlState='42000', errorCode=40000, errorMessage="Error while compiling statement: FAILED: ParseException line 3:17 cannot recognize input near 'orders' ';' '<EOF>' in table name"), operationHandle=None)


pyhive.exc.OperationalError(TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['Server-side error; please check HS2 logs.'], sqlState='42000', errorCode=40000, errorMessage="Error while compiling statement: FAILED: ParseException line 3:17 cannot recognize input near 'orders' ';' '<EOF>' in table name"), operationHandle=None))

In [ ]:
execute_query(cursor, '''
select concat(cust_id, '-', cust_name, '-', courier) from orders limit 5
''', True)

In [ ]:
execute_query(cursor, '''
SHOW PARTITIONS orders
''', True)

- RANK(): Quando você quer observar as lacunas causadas por empates.
- DENSE_RANK(): Quando você quer uma sequência contínua, mesmo com empates.
- ROW_NUMBER(): Quando precisa de números únicos para cada linha.
### Activity 2
1. Calculate the Maximum Buy rate.
2. Calculate average sell rate.
3. Calculate average buy rate.

In [33]:
execute_query(cursor, '''
CREATE TABLE IF NOT EXISTS market (
    ID VARCHAR(12),
    name STRING,
    buyprice FLOAT,
    sellprice FLOAT,
    units_purchsed INT,
    units_sold INT,
    year INT
) 
row format delimited fields terminated by ","
lines terminated by "\n" 
stored as textfile tblproperties("skip.header.line.count"="1")
''')
execute_query(cursor, '''
LOAD DATA LOCAL INPATH '/opt/hive/hive-assignment-2_2018new.txt' INTO TABLE market
''')
execute_query(cursor, '''
select * from market limit 5
''', True)

[('RNMM302671MC', 'Oil', 3.0, 20.0, 613, 452, 2002),
 ('UFBG160794BK', 'Face_Wash', 5.0, 21.0, 755, 471, 2001),
 ('RIEQ544903EP', 'Soap', 4.0, 18.0, 742, 492, 2009),
 ('HFJY800951JK', 'Tooth_Brush', 7.0, 16.0, 870, 497, 2005),
 ('EKMJ507797MG', 'Face_Wash', 6.0, 13.0, 848, 469, 2005)]

In [28]:

buy_rate = execute_query(cursor, '''
select name,units_purchsed from market order by units_purchsed limit 1
''', True)
print('buy rate = ',buy_rate[0])
avg_sell=execute_query(cursor, '''
select avg(units_sold) from market
''', True)
print('avg sell = ',avg_sell[0][0])
avg_buy = execute_query(cursor, '''
select avg(units_purchsed) from market
''', True)
print('avg buy = ',avg_buy[0][0])

buy rate =  ('Soap', 503)
avg sell =  449.4757834757835
avg buy =  755.4501424501425


In [51]:
execute_query(cursor, '''
drop table orders
''', False)